## Capstone notebook

This notebook is used for IBM capstone project.

In [1]:
# Database importing
import pandas as pd
import numpy as np

print("Hello Capstone Project Course!")


Hello Capstone Project Course!


## 1. Website data extraction

In [2]:
df = pd.read_html('List of postal codes of Canada_ M - Wikipedia.html')
df = df[0]


## 2. Drop address without Borough assigned

In [3]:
# 1. Drop not assined Borough.
i = df[df["Borough"] == "Not assigned"].index
df.drop(i, inplace = True)

## 3. Merge neighbourhood with same postal code

In [17]:
# 2. merge same postal code neighbourhood. we use postal code as index
cdf = pd.DataFrame(df.groupby('Postal Code')['Neighbourhood'].apply(','.join))

a = df.drop_duplicates(subset = ["Postal Code"])
a = a.drop('Neighbourhood',axis = 1)
a.set_index("Postal Code",inplace = True)

x = pd.concat([cdf, a], axis=1)
x.index.set_names(['Postal Code'],inplace = True)
x.reset_index(inplace = True)


### 3. replace not assigned Neighbourhood as Borough


In [18]:
# test only: x.loc[1:50,"Neighbourhood"]="Not assigned"
x.loc[x["Neighbourhood"]=="Not assigned","Neighbourhood"] = x.loc[x["Neighbourhood"]=="Not assigned","Borough"]


## 4. Show the cleaned dataframe

In [21]:
print(x)

print(x.shape)

    Postal Code                                      Neighbourhood  \
0           M1B                                     Malvern, Rouge   
1           M1C             Rouge Hill, Port Union, Highland Creek   
2           M1E                  Guildwood, Morningside, West Hill   
3           M1G                                             Woburn   
4           M1H                                          Cedarbrae   
..          ...                                                ...   
98          M9N                                             Weston   
99          M9P                                          Westmount   
100         M9R  Kingsview Village, St. Phillips, Martin Grove ...   
101         M9V  South Steeles, Silverstone, Humbergate, Jamest...   
102         M9W                Northwest, West Humber - Clairville   

         Borough  
0    Scarborough  
1    Scarborough  
2    Scarborough  
3    Scarborough  
4    Scarborough  
..           ...  
98          York  
99     